# TP2: Preprocessing and data visualization

## Winter 2023 - BIN710 Data Mining (UdeS)

Second assignement as part of the Data Mining class at UdeS.

Student name : Simon Lalonde

### Directory structure

├── package2.csv    ---> Data

├── product2.csv    ---> Data

├── tp1.ipynb   ---> Jupyter Notebook

└── TP1.pdf    ---> Tasks to complete

### Data
2 files for each dataset and both have the *byte signature*, meaning when compared byte by byte they are similar.

NDC = National Drug Code

### Metadata
Description for the 2 data files used : 
- [Product](https://www.fda.gov/drugs/drug-approvals-and-databases/ndc-product-file-definitions)
- [Package](https://www.fda.gov/drugs/drug-approvals-and-databases/ndc-package-file-definitions)

### Goal
Use preprocessing and data visualization techniques on FDA drugs databases

---

## 1, 2 and 3 : Data verification and cleaning for individual tables (coherence, types, redundency etc.)

Importing all required libraries and modules. Reading file to dataframe with proper encoding

In [297]:
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer

In [2]:
root_dir = Path.cwd()
pack = pd.read_csv(root_dir / "package2.csv", delimiter=";")
prod = pd.read_csv(root_dir / "product2.csv", delimiter=";", encoding="ISO-8859-1")    # Latin-1 encoding



### Exploring and cleaning the product data table

In [3]:
prod.head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,NaN,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710,NaN,NDA,NDA018781,10,WATER,1,mL/mL,NaN,NaN,N,20201231.0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20211231.0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0


In [4]:
print(f"Product dataframe has {prod.shape[0]} objects and {prod.shape[1]} columns.") 

Product dataframe has 93238 objects and 20 columns.


In [5]:
print(f" There are {len(prod.dtypes[prod.dtypes != 'object'])} numerical columns :\n") 
print(prod.dtypes[prod.dtypes != "object"].index.to_list())

 There are 3 numerical columns :

['STARTMARKETINGDATE', 'ENDMARKETINGDATE', 'LISTING_RECORD_CERTIFIED_THROUGH']


In [6]:
print(f" There are {len(prod.dtypes[prod.dtypes == 'object'])} non-numerical columns :\n") 
print(prod.dtypes[prod.dtypes == "object"].index.to_list())

 There are 17 non-numerical columns :

['PRODUCTID', 'PRODUCTNDC', 'PRODUCTTYPENAME', 'PROPRIETARYNAME', 'PROPRIETARYNAMESUFFIX', 'NONPROPRIETARYNAME', 'DOSAGEFORMNAME', 'ROUTENAME', 'MARKETINGCATEGORYNAME', 'APPLICATIONNUMBER', 'LABELERNAME', 'SUBSTANCENAME', 'ACTIVE_NUMERATOR_STRENGTH', 'ACTIVE_INGRED_UNIT', 'PHARM_CLASSES', 'DEASCHEDULE', 'NDC_EXCLUDE_FLAG']


Looking at null/missing values for each feature. We see some feature with very high missing vals such as ProrietaryNameSuffix, EndMarketingDate and DeaSchedule. It makes sense since these are not missing per-se but information on the object itself (No need for suffix on Rx, does have a known end marketing date and no classification of dependency potential respectively)

In [7]:
prod.isnull().sum()

PRODUCTID                            1560
PRODUCTNDC                              0
PRODUCTTYPENAME                         0
PROPRIETARYNAME                         6
PROPRIETARYNAMESUFFIX               83075
NONPROPRIETARYNAME                      4
DOSAGEFORMNAME                          0
ROUTENAME                            1932
STARTMARKETINGDATE                      0
ENDMARKETINGDATE                    88915
MARKETINGCATEGORYNAME                   0
APPLICATIONNUMBER                   13097
LABELERNAME                             0
SUBSTANCENAME                        2309
ACTIVE_NUMERATOR_STRENGTH            2309
ACTIVE_INGRED_UNIT                   2309
PHARM_CLASSES                       50984
DEASCHEDULE                         88815
NDC_EXCLUDE_FLAG                        0
LISTING_RECORD_CERTIFIED_THROUGH     4325
dtype: int64

ProductID is supposed to be composed of ProductNDC and other information. Let's look if if's true

In [8]:
print(f"Num objects with no NA in ProdID/NDC : {len(prod[['PRODUCTID', 'PRODUCTNDC']].dropna())}")

Num objects with no NA in ProdID/NDC : 91678


In [9]:
print(f"Num objects with NDC code within ID col : {prod[['PRODUCTID', 'PRODUCTNDC']].dropna().apply(lambda x : x.PRODUCTNDC in x.PRODUCTID, axis=1).sum()}")

Num objects with NDC code within ID col : 91165


In [10]:
prod[prod['PRODUCTID'].notna()].head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
20,0002-3251_67a53369-eead-4f2c-afe9-f3274899c47e,0002-3251,HUMAN PRESCRIPTION DRUG,Strattera,NaN,Atomoxetine hydrochloride,CAPSULE,ORAL,20050214,NaN,NDA,NDA021411,10,ATOMOXETINE HYDROCHLORIDE,100,mg/1,"Norepinephrine Reuptake Inhibitor [EPC],Norepi...",NaN,N,20211231.0
21,0002-3270_06e2a1f2-459c-45aa-9341-54e36f7726a7,0002-3270,HUMAN PRESCRIPTION DRUG,Cymbalta,NaN,Duloxetine hydrochloride,"CAPSULE, DELAYED RELEASE",ORAL,20100115,NaN,NDA,NDA021427,10,DULOXETINE HYDROCHLORIDE,60,mg/1,"Norepinephrine Uptake Inhibitors [MoA],Seroton...",NaN,N,20201231.0
22,0002-4112_d561034d-ea58-45fe-9d07-2e9eba98c2e4,0002-4112,HUMAN PRESCRIPTION DRUG,Zyprexa,NaN,Olanzapine,TABLET,ORAL,19970623,NaN,NDA,NDA020592,10,OLANZAPINE,2.5,mg/1,Atypical Antipsychotic [EPC],NaN,N,20201231.0
23,0002-4115_d561034d-ea58-45fe-9d07-2e9eba98c2e4,0002-4115,HUMAN PRESCRIPTION DRUG,Zyprexa,NaN,Olanzapine,TABLET,ORAL,19961001,NaN,NDA,NDA020592,10,OLANZAPINE,5,mg/1,Atypical Antipsychotic [EPC],NaN,N,20201231.0
24,0002-4116_d561034d-ea58-45fe-9d07-2e9eba98c2e4,0002-4116,HUMAN PRESCRIPTION DRUG,Zyprexa,NaN,Olanzapine,TABLET,ORAL,19961001,NaN,NDA,NDA020592,10,OLANZAPINE,7.5,mg/1,Atypical Antipsychotic [EPC],NaN,N,20201231.0


In [11]:
id_ndc_incoherent = prod[prod['PRODUCTID'].notna()][prod[['PRODUCTID', 'PRODUCTNDC']].dropna().apply(lambda x : x.PRODUCTNDC not in x.PRODUCTID, axis=1)]
print(f"Num objects with incoherent ID and NDC : {len(id_ndc_incoherent)}")

Num objects with incoherent ID and NDC : 513


In [12]:
id_ndc_incoherent.head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
159,0006-0005_0c7a3452-ecb2-4f66-ad52-94f8eaf8cde8,05-juin,HUMAN PRESCRIPTION DRUG,BELSOMRA,NaN,suvorexant,"TABLET, FILM COATED",ORAL,20140829,NaN,NDA,NDA204569,10,SUVOREXANT,5,mg/1,"Orexin Receptor Antagonist [EPC],Orexin Recept...",CIV,N,20211231.0
160,0006-0019_54e9c31a-9429-4842-b2d6-0cc1e5ad613c,19-juin,HUMAN PRESCRIPTION DRUG,PRINIVIL,NaN,lisinopril,TABLET,ORAL,19871229,NaN,NDA,NDA019558,10,LISINOPRIL,5,mg/1,"Angiotensin Converting Enzyme Inhibitor [EPC],...",NaN,N,20201231.0
310,0009-0003_67759a7c-ea06-4151-87e1-a301c44d67cd,03-sept,HUMAN PRESCRIPTION DRUG,SOLU-MEDROL,NaN,methylprednisolone sodium succinate,"INJECTION, POWDER, FOR SOLUTION",INTRAMUSCULAR; INTRAVENOUS,19590402,NaN,NDA,NDA011856,Pharmacia and Upjohn Company LLC,METHYLPREDNISOLONE SODIUM SUCCINATE,500,mg/4mL,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NaN,N,20201231.0
311,0009-0005_c9aa26c1-05c3-479c-90eb-63b2181c5e7e,05-sept,HUMAN PRESCRIPTION DRUG,Solu-Cortef,NaN,hydrocortisone sodium succinate,"INJECTION, POWDER, FOR SOLUTION",INTRAMUSCULAR; INTRAVENOUS,19550427,NaN,NDA,NDA009866,Pharmacia and Upjohn Company LLC,HYDROCORTISONE SODIUM SUCCINATE,1000,mg/8mL,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NaN,N,20201231.0
312,0009-0011_c9aa26c1-05c3-479c-90eb-63b2181c5e7e,11-sept,HUMAN PRESCRIPTION DRUG,Solu-Cortef,NaN,hydrocortisone sodium succinate,"INJECTION, POWDER, FOR SOLUTION",INTRAMUSCULAR; INTRAVENOUS,19550427,NaN,NDA,NDA009866,Pharmacia and Upjohn Company LLC,HYDROCORTISONE SODIUM SUCCINATE,100,mg/2mL,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NaN,N,20201231.0


In [13]:
# List of possible vals
id_ndc_incoherent["PRODUCTNDC"].unique()

array(['05-juin', '19-juin', '03-sept', '05-sept', '11-sept', '12-sept',
       '13-sept', '16-sept', '17-sept', '18-sept', '20-sept', '22-sept',
       '29-sept', 'OTC MONOGRAPH NOT FINAL', 'NDA', 'OTC MONOGRAPH FINAL',
       'UNAPPROVED HOMEOPATHIC', 'UNAPPROVED MEDICAL GAS',
       'UNAPPROVED DRUG OTHER', 'ANDA', 'NDA AUTHORIZED GENERIC', 'BLA'],
      dtype=object)

We could drop the problematic entries with:

`prod = prod.drop(id_ndc_incoherent.index)` 


In [14]:
prod.iloc[id_ndc_incoherent.index, :].head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
159,0006-0005_0c7a3452-ecb2-4f66-ad52-94f8eaf8cde8,05-juin,HUMAN PRESCRIPTION DRUG,BELSOMRA,NaN,suvorexant,"TABLET, FILM COATED",ORAL,20140829,NaN,NDA,NDA204569,10,SUVOREXANT,5,mg/1,"Orexin Receptor Antagonist [EPC],Orexin Recept...",CIV,N,20211231.0
160,0006-0019_54e9c31a-9429-4842-b2d6-0cc1e5ad613c,19-juin,HUMAN PRESCRIPTION DRUG,PRINIVIL,NaN,lisinopril,TABLET,ORAL,19871229,NaN,NDA,NDA019558,10,LISINOPRIL,5,mg/1,"Angiotensin Converting Enzyme Inhibitor [EPC],...",NaN,N,20201231.0
310,0009-0003_67759a7c-ea06-4151-87e1-a301c44d67cd,03-sept,HUMAN PRESCRIPTION DRUG,SOLU-MEDROL,NaN,methylprednisolone sodium succinate,"INJECTION, POWDER, FOR SOLUTION",INTRAMUSCULAR; INTRAVENOUS,19590402,NaN,NDA,NDA011856,Pharmacia and Upjohn Company LLC,METHYLPREDNISOLONE SODIUM SUCCINATE,500,mg/4mL,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NaN,N,20201231.0
311,0009-0005_c9aa26c1-05c3-479c-90eb-63b2181c5e7e,05-sept,HUMAN PRESCRIPTION DRUG,Solu-Cortef,NaN,hydrocortisone sodium succinate,"INJECTION, POWDER, FOR SOLUTION",INTRAMUSCULAR; INTRAVENOUS,19550427,NaN,NDA,NDA009866,Pharmacia and Upjohn Company LLC,HYDROCORTISONE SODIUM SUCCINATE,1000,mg/8mL,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NaN,N,20201231.0
312,0009-0011_c9aa26c1-05c3-479c-90eb-63b2181c5e7e,11-sept,HUMAN PRESCRIPTION DRUG,Solu-Cortef,NaN,hydrocortisone sodium succinate,"INJECTION, POWDER, FOR SOLUTION",INTRAMUSCULAR; INTRAVENOUS,19550427,NaN,NDA,NDA009866,Pharmacia and Upjohn Company LLC,HYDROCORTISONE SODIUM SUCCINATE,100,mg/2mL,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NaN,N,20201231.0


In [15]:
# prod = prod.drop(columns="PRODUCTNDC")

**ProductTypeName FDA labels verification**

In [16]:
content_type_label = pd.read_html("https://www.fda.gov/industry/structured-product-labeling-resources/document-type-including-content-labeling-type")[0]["LOINC Name"]


In [17]:
content_type_label = content_type_label.str.replace("LABEL", "")
content_type_label = content_type_label.str.rstrip().to_list()

In [18]:
print(f'{len([label for label in prod["PRODUCTTYPENAME"].unique() if label not in content_type_label])} producttype categories not in official FDA repo')

0 producttype categories not in official FDA repo


**Dosage form FDA codes verification**

In [19]:
dosage_form_codes = pd.read_html("https://www.fda.gov/industry/structured-product-labeling-resources/dosage-forms")[0]
dosage_form_codes = dosage_form_codes["SPL Acceptable Term"].to_list()


In [20]:
[label for label in prod["DOSAGEFORMNAME"].unique() if label not in dosage_form_codes]
print(f"{len([label for label in prod['DOSAGEFORMNAME'].unique() if label not in dosage_form_codes])} DosageForm categories not in official FDA repo codes")

0 DosageForm categories not in official FDA repo codes


**RouteName FDA codes verification**

In [21]:
routename_codes = pd.read_html("https://www.fda.gov/industry/structured-product-labeling-resources/route-administration")[0]
routename_codes = routename_codes["SPL Acceptable Term"].to_list()


In [22]:
print(f'{len([label for label in prod["ROUTENAME"].dropna().unique() if label not in routename_codes])} RouteNames not listed in official FDA repo')

127 RouteNames not listed in official FDA repo


In [23]:
# Example of multiple categories for RouteName
print([label for label in prod["ROUTENAME"].dropna().unique() if label not in routename_codes][:10])

['INTRAMUSCULAR; SUBCUTANEOUS', 'INTRAVENOUS; SUBCUTANEOUS', 'INTRA-ARTICULAR; INTRAMUSCULAR', 'INTRA-ARTICULAR; INTRALESIONAL', 'INTRAMUSCULAR; INTRAVENOUS', 'INTRALESIONAL; INTRAMUSCULAR; INTRASYNOVIAL; SOFT TISSUE', 'INTRAMUSCULAR; INTRAVENOUS; SUBCONJUNCTIVAL', 'INTRA-ARTICULAR; INTRALESIONAL; INTRAMUSCULAR; SOFT TISSUE', 'INTRAVASCULAR; INTRAVENOUS', 'INTRA-ARTERIAL; INTRAVENOUS']


In [24]:
# Check for lowercase
len(prod["ROUTENAME"].dropna()[prod["ROUTENAME"].str.islower().dropna()])

0

We see it's the objects with multiple categories that have those special labels. This will get fixed with get_dummies method or OneHotEncoding

In [25]:
# Pandas get_dummies example
prod["ROUTENAME"].str.upper().str.get_dummies().head()

,AURICULAR (OTIC),BUCCAL,BUCCAL; DENTAL; TOPICAL,BUCCAL; SUBLINGUAL,BUCCAL; VAGINAL,CUTANEOUS,CUTANEOUS; EXTRACORPOREAL,CUTANEOUS; EXTRACORPOREAL; TOPICAL; VAGINAL,CUTANEOUS; EXTRACORPOREAL; VAGINAL,CUTANEOUS; INTRADERMAL; SUBCUTANEOUS,...,TOPICAL,TOPICAL; TOPICAL,TOPICAL; TOPICAL; TOPICAL,TOPICAL; TRANSDERMAL,TOPICAL; VAGINAL,TRANSDERMAL,TRANSMUCOSAL,URETERAL,URETHRAL,VAGINAL
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
#ohe example
enc = OneHotEncoder()
enc.fit(pd.DataFrame(prod["ROUTENAME"].str.upper()))
enc.categories_[0][:5]

array(['AURICULAR (OTIC)', 'BUCCAL', 'BUCCAL; DENTAL; TOPICAL',
       'BUCCAL; SUBLINGUAL', 'BUCCAL; VAGINAL'], dtype=object)

Some categories are repeated multiple times with the same routename which does not make sense

In [27]:
prod[prod["ROUTENAME"] == "TOPICAL; TOPICAL; TOPICAL"]

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
74661,68466-0002_b9ea1370-4627-458d-8460-3c1fa0a56f48,68466-0002,HUMAN OTC DRUG,Sports For Trauma Gel,NaN,"Bellis Perennis, Hypericum Perfomatum,Toxicode...",GEL,TOPICAL; TOPICAL; TOPICAL,20040701,NaN,UNAPPROVED HOMEOPATHIC,NaN,"Schwabe Mexico, S.A. de C.V.",BELLIS PERENNIS; HYPERICUM PERFORATUM; TOXICOD...,1; 2; 3; 1,[hp_X]/71g; [hp_X]/71g; [hp_X]/71g; [hp_X]/71g,NaN,NaN,N,20201231.0


In [28]:
# Find the elements with multiple RouteNames
prod["ROUTENAME"][prod["ROUTENAME"].str.split("; ").str.len() > 1].head()

49    INTRAMUSCULAR; SUBCUTANEOUS
52      INTRAVENOUS; SUBCUTANEOUS
55      INTRAVENOUS; SUBCUTANEOUS
70    INTRAMUSCULAR; SUBCUTANEOUS
71    INTRAMUSCULAR; SUBCUTANEOUS
Name: ROUTENAME, dtype: object

In [29]:
# Find all the elements with RouteName repetitions
print(f'Num of RouteName repetitions : {len(prod[prod["ROUTENAME"].str.split("; ").str.len() > 1][prod["ROUTENAME"][prod["ROUTENAME"].str.split("; ").str.len() > 1].str.split("; ").apply(lambda x : set(x)).str.len() == 1])}')
prod[prod["ROUTENAME"].str.split("; ").str.len() > 1][prod["ROUTENAME"][prod["ROUTENAME"].str.split("; ").str.len() > 1].str.split("; ").apply(lambda x : set(x)).str.len() == 1].head()

Num of RouteName repetitions : 32


,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
9927,0527-1109_458431c7-41c7-48f4-a8f6-b6b6ccc7cbe2,0527-1109,HUMAN PRESCRIPTION DRUG,Isoniazid,NaN,Isoniazid,TABLET,ORAL; ORAL,20131010,NaN,ANDA,ANDA089776,"Lannett Company, Inc.",ISONIAZID,300,mg/1,Antimycobacterial [EPC],NaN,N,20211231.0
11121,0615-8061_4643015f-3f68-4ecd-909f-85e3fd2c8549,0615-8061,HUMAN PRESCRIPTION DRUG,Lisinopril,NaN,Lisinopril,TABLET,ORAL; ORAL,20111101,20200930.0,ANDA,ANDA076180,"NCS HealthCare of KY, Inc dba Vangard Labs",LISINOPRIL,2.5,mg/1,"Angiotensin Converting Enzyme Inhibitor [EPC],...",NaN,N,NaN
12307,0869-0012_2fd9a395-b322-45b0-b568-71b98e581ae4,0869-0012,HUMAN OTC DRUG,Vitamin A D,NaN,"Lanolin, Petrolatum",OINTMENT,TOPICAL; TOPICAL,20130701,NaN,OTC MONOGRAPH FINAL,part347,Vi-Jon,LANOLIN; PETROLATUM,133; 459,mg/g; mg/g,NaN,NaN,N,20211231.0
17328,16714-114_6ae8605d-16ec-9ea6-8389-ba144c924ee1,16714-114,HUMAN PRESCRIPTION DRUG,Fluoxetine hydrochloride,NaN,Fluoxetine hydrochloride,"TABLET, FILM COATED",ORAL; ORAL,20190918,NaN,ANDA,ANDA211721,NorthStar Rx LLC,FLUOXETINE HYDROCHLORIDE,60,mg/1,"Serotonin Reuptake Inhibitor [EPC],Serotonin U...",NaN,N,20201231.0
29347,43598-632_b7779005-0433-747c-3ea9-16e6d45b6cee,43598-632,HUMAN PRESCRIPTION DRUG,Fluoxetine hydrochloride,NaN,Fluoxetine hydrochloride,"TABLET, FILM COATED",ORAL; ORAL,20190128,NaN,ANDA,ANDA211721,Dr. Reddy's Laboratories Inc.,FLUOXETINE HYDROCHLORIDE,60,mg/1,"Serotonin Reuptake Inhibitor [EPC],Serotonin U...",NaN,N,20201231.0


Shrink to a single categorie for the 32 objects with repetitions

In [30]:
prod.loc[
    prod["ROUTENAME"][prod["ROUTENAME"].str.split("; ").str.len() > 1].str.split("; ").apply(lambda x: set(x))[prod["ROUTENAME"][prod["ROUTENAME"].str.split("; ").str.len() > 1].str.split("; ").apply(lambda x: set(x)).str.len() == 1].index,
    "ROUTENAME"
] = prod["ROUTENAME"][prod["ROUTENAME"].str.split("; ").str.len() > 1].str.split("; ").apply(lambda x: list(set(x))[0])[prod["ROUTENAME"][prod["ROUTENAME"].str.split("; ").str.len() > 1].str.split("; ").apply(lambda x: set(x)).str.len() == 1]
# [prod["ROUTENAME"][prod["ROUTENAME"].str.split("; ").str.len() > 1].str.split("; ")]
# .apply(lambda x : set(x)).str.len() == 1]

**Verifying DateType attributes (START/END/Listing_Record_Certified_Through)**

First let's transform the data from int to datetime format

In [31]:
prod["STARTMARKETINGDATE"] = pd.to_datetime(prod["STARTMARKETINGDATE"], format="%Y%m%d")

In [32]:
print(f'Date range from {min(prod["STARTMARKETINGDATE"])} to {max(prod["STARTMARKETINGDATE"])} for marketing start : OK')

Date range from 1900-01-01 00:00:00 to 2020-02-14 00:00:00 for marketing start : OK


Error for 3031 year in timestamp of EndMarketingDate. Let's fix it to 2031

In [33]:
# pd.to_datetime(prod["ENDMARKETINGDATE"], format="%Y%m%d")
prod["ENDMARKETINGDATE"].sort_values(ascending=False).head()

29503    30310209.0
65640    20390831.0
46709    20380131.0
89575    20331010.0
89576    20331010.0
Name: ENDMARKETINGDATE, dtype: float64

In [34]:
prod.loc[prod["ENDMARKETINGDATE"] > 20500000, ["ENDMARKETINGDATE"]] = 30310209.0 - 10000000

In [35]:
print(prod.iloc[29503]["ENDMARKETINGDATE"])    # Check replacement

20310209.0


In [36]:
# Converting the actual data
prod["ENDMARKETINGDATE"] = pd.to_datetime(prod["ENDMARKETINGDATE"], format="%Y%m%d")

In [37]:
print(f'Date range from {min(prod["ENDMARKETINGDATE"].dropna())} to {max(prod["ENDMARKETINGDATE"].dropna())} for marketing end : OK')

Date range from 2020-02-15 00:00:00 to 2039-08-31 00:00:00 for marketing end : OK


No objects with incongruent start/end date combinations

In [38]:
print(f'Number of objects with enddates greather than start dates : {len(prod[prod["ENDMARKETINGDATE"] < prod["STARTMARKETINGDATE"]])}')

Number of objects with enddates greather than start dates : 0


Listing records to date time, no incongruencies

In [39]:
prod["LISTING_RECORD_CERTIFIED_THROUGH"] = pd.to_datetime(prod["LISTING_RECORD_CERTIFIED_THROUGH"], format="%Y%m%d")

In [40]:
print(f'Date range from {min(prod["LISTING_RECORD_CERTIFIED_THROUGH"].dropna())} to {max(prod["LISTING_RECORD_CERTIFIED_THROUGH"].dropna())} for marketing end : OK')

Date range from 2020-12-31 00:00:00 to 2021-12-31 00:00:00 for marketing end : OK


In [41]:
print(f'Number of objects with listing certified greather than start dates : {len(prod[prod["LISTING_RECORD_CERTIFIED_THROUGH"] < prod["STARTMARKETINGDATE"]])}')

Number of objects with listing certified greather than start dates : 0


**Let's transform the date columns with massive amount of NaN because the dates are not missing but not determined**

In [42]:
na75_threshold = 0.75*len(prod)
print(na75_threshold)

69928.5


In [43]:
print(prod.isnull().sum()[prod.isnull().sum() > na75_threshold].index.to_list())

['PROPRIETARYNAMESUFFIX', 'ENDMARKETINGDATE', 'DEASCHEDULE']


**DEASchedule verification codes**

In [44]:
deas_codes = ["CI", "CII", "CIII", "CIV", "CV"]
print(len([label for label in prod["DEASCHEDULE"].dropna().unique() if label not in deas_codes]))

0


Let's replace it with a binary feature anyway since most of them are not determined

Replacing ENDMARKETINGDATE with determined/notdetermined binary

In [45]:
# Testing the logic
pd.DataFrame(np.where(prod["ENDMARKETINGDATE"].notnull(), 1, 0)).value_counts()

0    88915
1     4323
dtype: int64

In [46]:
pd.DataFrame(np.where(prod["DEASCHEDULE"].notnull(), 1, 0)).value_counts()

0    88815
1     4423
dtype: int64

In [47]:
# Replacing values
prod["ENDMARKETINGDATE_determined"] = np.where(prod["ENDMARKETINGDATE"].notnull(), 1, 0)
prod["DEASCHEDULE_determined"] = np.where(prod["DEASCHEDULE"].notnull(), 1, 0)

In [48]:
prod = prod.drop(columns=["ENDMARKETINGDATE", "DEASCHEDULE"])

**Application Number verification to startwith FDA reference codes**

In [49]:
# Does not matc prefix NDA / ANDA / BLA or partXXXX in ApplicationNumber
prod["APPLICATIONNUMBER"].dropna()[prod["APPLICATIONNUMBER"].dropna().str.match("^[^NDA|^ANDA|^BLA|^part]")]

26428    333D
Name: APPLICATIONNUMBER, dtype: object

Let's remove that object

In [50]:
prod = prod.drop(prod["APPLICATIONNUMBER"].dropna()[prod["APPLICATIONNUMBER"].dropna().str.match("^[^NDA|^ANDA|^BLA|^part]")].index)

In [51]:
prod.head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH,ENDMARKETINGDATE_determined,DEASCHEDULE_determined
0,NaN,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,1987-07-10,NDA,NDA018781,10,WATER,1,mL/mL,NaN,N,2020-12-31,0,0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,2012-06-01,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",N,2021-12-31,0,0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,2014-09-18,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",N,2020-12-31,0,0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,2014-09-18,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",N,2020-12-31,0,0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,2018-09-27,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,N,2020-12-31,0,0


**Verify if ApplicationNumber prefix and MarketingCategoryName are identical**

In [52]:
prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].isnull().sum()

MARKETINGCATEGORYNAME        0
APPLICATIONNUMBER        13097
dtype: int64

Let's look at the values for MarketingCategoryName when ApplicationNumber is null

In [53]:
prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]][prod["APPLICATIONNUMBER"].isnull()].head()

,MARKETINGCATEGORYNAME,APPLICATIONNUMBER
738,UNAPPROVED DRUG OTHER,NaN
750,UNAPPROVED DRUG OTHER,NaN
2262,UNAPPROVED DRUG OTHER,NaN
2307,UNAPPROVED DRUG OTHER,NaN
2822,UNAPPROVED DRUG OTHER,NaN


For now we leave the NaN fields in ApplicationNumber as empty

In [54]:
prod["MARKETINGCATEGORYNAME"][prod["APPLICATIONNUMBER"].isnull()].unique()

array(['UNAPPROVED DRUG OTHER', 'UNAPPROVED HOMEOPATHIC',
       'UNAPPROVED MEDICAL GAS',
       'UNAPPROVED DRUG FOR USE IN DRUG SHORTAGE'], dtype=object)

List of the categories where ApplicationNumber prefix does not match MarketingCategoryName show that NDA and AND have some mismatch between the 2 features

In [55]:
print(prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna()[prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna().apply(lambda x : x["MARKETINGCATEGORYNAME"] not in x["APPLICATIONNUMBER"], axis=1)]["MARKETINGCATEGORYNAME"].unique())

['OTC MONOGRAPH NOT FINAL' 'OTC MONOGRAPH FINAL' 'NDA AUTHORIZED GENERIC'
 'NDA' 'ANDA']


OK for OTC MONOGRAPH + NDA AUTHORIZED GENERIC entries

In [56]:
# Entries with NDA AUTHORIZED GENERIC AND NOT STARTING WITH NDAXXXXX IN APPLICATION NUMBER
prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna()[prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna().apply(lambda x : x["MARKETINGCATEGORYNAME"] not in x["APPLICATIONNUMBER"] and x["MARKETINGCATEGORYNAME"] == "NDA AUTHORIZED GENERIC", axis=1)]["APPLICATIONNUMBER"].str.contains("^[^NDA]").sum()

0

Incongruencies for ANDA and NDA labelled objects in ApplicationNumber

In [57]:
# For ANDA in Marketing
print(f'{len(prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna()[prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna().apply(lambda x : x["MARKETINGCATEGORYNAME"] not in x["APPLICATIONNUMBER"] and x["MARKETINGCATEGORYNAME"] == "ANDA", axis=1)])} mislabelled ANDA samples')
prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna()[prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna().apply(lambda x : x["MARKETINGCATEGORYNAME"] not in x["APPLICATIONNUMBER"] and x["MARKETINGCATEGORYNAME"] == "ANDA", axis=1)]


12 mislabelled ANDA samples


,MARKETINGCATEGORYNAME,APPLICATIONNUMBER
9209,ANDA,BA740193
9228,ANDA,BA720563
9229,ANDA,BA720562
16915,ANDA,BA010228
16916,ANDA,BA010228
16920,ANDA,BA125608
16921,ANDA,BA125608
16923,ANDA,BA010228
28399,ANDA,BA110057
41792,ANDA,BA740193


In [58]:
# NDA mislabelled
print(f'{len(prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna()[prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna().apply(lambda x : x["MARKETINGCATEGORYNAME"] not in x["APPLICATIONNUMBER"] and x["MARKETINGCATEGORYNAME"] == "NDA", axis=1)])} mislabelled NDA samples')
prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna()[prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna().apply(lambda x : x["MARKETINGCATEGORYNAME"] not in x["APPLICATIONNUMBER"] and x["MARKETINGCATEGORYNAME"] == "NDA", axis=1)]

126 mislabelled NDA samples


,MARKETINGCATEGORYNAME,APPLICATIONNUMBER
5955,NDA,BN890105
8905,NDA,BN070012
8966,NDA,BN200952
13099,NDA,BN160918
13100,NDA,BN160918
...,...,...
50017,NDA,BN980123
50020,NDA,BN000127
50021,NDA,BN000127
50022,NDA,BN000127


Remove them from prod dataframe

In [59]:
prod = prod.drop(prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna()[prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna().apply(lambda x : x["MARKETINGCATEGORYNAME"] not in x["APPLICATIONNUMBER"] and x["MARKETINGCATEGORYNAME"] == "ANDA", axis=1)].index)
prod = prod.drop(prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna()[prod[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]].dropna().apply(lambda x : x["MARKETINGCATEGORYNAME"] not in x["APPLICATIONNUMBER"] and x["MARKETINGCATEGORYNAME"] == "NDA", axis=1)].index)


ProprietaryName seems all over the place and there are no standard format/regulations for this feature. Maybe delete it later on since it might not bring valuable information. At least convert to upper/lowercase before OneHotEncoding

In [60]:
# Example propname
print(prod["PROPRIETARYNAME"].sort_values().unique()[:50])

['(CHLOROPROCAINE HCI' '.Insulin Aspart Protamine and Insulin Aspart'
 '0.9% SODIUM CHLORIDE' '02 CUSHION SPF45' '1 Bladder' '1 Detoxification'
 '1% Hydrocortisone' '1% LIDOCAINE HCI'
 '1.8OZ HAND SANITIZER WITH CLIP -ASSORTED'
 '1.8oz Armstrong Hand Sanitizer with Aloe Vera and Vitamin E'
 '10 Armani Prima Control Glow Moisturizer SBS SPF 35' '10 PARASITE DETOX'
 '10 Parasite Detox' '10 TREE MIX' '100% Pure Yerba Mate MIst'
 '1000 Roses CC Color Plus Correct Sheer Tan SPF 30'
 '1000 Roses CC Color plus Correct Sheer Nude SPF 30'
 '1000 Roses Daily Shade Facial SPF 18' '1012 Antimicrobial'
 '10g Colgate plus Toothbrush Kit'
 '10g Colgate plus Toothbrush plus Floss Kit' '11 Tree Pollen Mix'
 '111 Medco Benzoyl Peroxide' '12 Hour Nasal' '12 Hour Nasal Decongestant'
 '12 Hour Original Nasal Decongestant' '12 hour allergy and congestion'
 '12 hour allergy d' '12 hour decongestant'
 '12HR Allergy and Congestion Relief' '16OZ HYDORGEN PEROXIDE'
 '1ST MEDXPATCH' '1st RELIEF TOPICAL' '2 Cockro

In [61]:
prod["PROPRIETARYNAME"] = prod["PROPRIETARYNAME"].str.upper()

There are a lot of similar companies with just slight variations in their name. It might lead to more confusion or decreasing the model metrics such as Accuracy and F1 scores.

For example look at the 5 different name variations for 7-eleven

In [62]:
prod["LABELERNAME"].sort_values().unique()[:50]

array(['- INDUSTRIAL WELDING SUPPLY CO. OF HARVEY, INC.',
       "-L'Oreal USA Products Inc", '.Cardinal Health',
       '1 Veterans Health', '10', '101196749', '111 Medco',
       '1ST MEDX LLC', '1st Class Pharmaceuticals, Inc.', '2 Transform',
       '20Lighter, LLC.', '21st Century Designer Health Products',
       '21st Century Formulations', '21st Century Homeopathics',
       '21st Century Homeopathics, Inc', '2xl Corporation', '3014704014',
       '3D Imaging Drug Design and Development LLC', '3LAB', '3LAB, Inc',
       '3LAB, Inc.', '3M Company', '3M ESPE Dental Products',
       '3M Health Care', '4E Global, S.A.P.I. de C.V',
       '4Life Research USA, LLC', '4e Brands Northamerica LLC',
       '4sport D.o.o.', '5 SENCO Inc.', '60 Degrees Pharmaceuticals, LLC',
       '65364-110', '7-ELEVEN', '7-ELEVEN, INC.', '7-Eleven',
       '7-Eleven Inc', '7-Eleven Inc.', '7-Eleven, Inc.', '7-Select, Inc',
       '7-eleven', '714 Essentials LTD', '7T Pharma LLC',
       '8.0 IDEAL BALA

In [63]:
len(prod["LABELERNAME"].unique())

6599

Replace to uppercase to control for case-sensitive str

In [64]:
prod["LABELERNAME"] = prod["LABELERNAME"].str.upper()

Let's drop the ProprietaryName since it there is no standard and so many different unique categories it won't bring anything good to the model

In [65]:
len(prod["PROPRIETARYNAME"].unique())

29723

In [66]:
prod = prod.drop(columns="PROPRIETARYNAME")

Suffix feature will mostly not contribute a lot of relevant information since there are not really a standard and many incongruency in naming

In [67]:
print(f"{len(prod['PROPRIETARYNAMESUFFIX'].dropna())} values for PPTsuffix with {len(prod['PROPRIETARYNAMESUFFIX'].dropna().unique())} unique categories")

10157 values for PPTsuffix with 4019 unique categories


In [68]:
# See a few examples
print(prod['PROPRIETARYNAMESUFFIX'].dropna().unique()[:10])

['Zydis ' 'Mix75/25 ' 'Mix50/50 ' 'Intramuscular ' 'Relprevv ' 'KwikPen '
 ' Junior KwikPen ' ' Tempo Pen ' 'R ' 'N ']


Transform to a `PROPRIETARYNAMESUFFIX_determined` attribute to account for that high proportion of missing data

In [69]:
pd.DataFrame(np.where(prod["PROPRIETARYNAMESUFFIX"].notnull(), 1, 0)).value_counts()

0    82942
1    10157
dtype: int64

In [70]:
# Replacing vals
prod["PROPRIETARYNAMESUFFIX_determined"] = np.where(prod["PROPRIETARYNAMESUFFIX"].notnull(), 1, 0)

In [71]:
# Droping original feature
prod = prod.drop(columns="PROPRIETARYNAMESUFFIX")

**No other cleanup/checkup except to convert to uppercase/lowercase before OneHotEncoding of NonProprietaryName to avoid repeated categories. NonProprietaryName is the list of activate ingredients and has a standard naming convention so it will be usefull for the classification task**

In [72]:
prod["NONPROPRIETARYNAME"] = prod["NONPROPRIETARYNAME"].str.upper()

In [73]:
prod["NONPROPRIETARYNAME"].head()

0             DILUENT
1    FLORBETAPIR F 18
2         DULAGLUTIDE
3         DULAGLUTIDE
4        GALCANEZUMAB
Name: NONPROPRIETARYNAME, dtype: object

**SubstanceName, ACTIVATE_NUMERATOR_STRENGTH (which on the FDA website is labelled as StrengthNumber) and ACTIVE_INGRED_UNIT are linked features. They also can have multiple values each when split by a semi colon. Let's look if the number of multiple values match its counterpart in each feature**

Looking at the frequency of number of elements per value. They look similar.

In [74]:
prod["SUBSTANCENAME"].str.split(";").str.len().astype(str).value_counts()[:6]

1.0    69757
2.0     9480
3.0     4497
4.0     2677
nan     2252
5.0     1124
Name: SUBSTANCENAME, dtype: int64

In [75]:
prod["ACTIVE_NUMERATOR_STRENGTH"].str.split(";").str.len().astype(str).value_counts()[:6]

1.0    69757
2.0     9481
3.0     4496
4.0     2677
nan     2252
5.0     1124
Name: ACTIVE_NUMERATOR_STRENGTH, dtype: int64

In [76]:
prod["ACTIVE_INGRED_UNIT"].str.split(";").str.len().astype(str).value_counts()[:6]

1.0    69757
2.0     9481
3.0     4496
4.0     2677
nan     2252
5.0     1124
Name: ACTIVE_INGRED_UNIT, dtype: int64

Looking at differences in amount of categories per objects for SubstanceName and ACTIVE_NUMERATOR_STRENGTH

In [77]:
prod[prod["SUBSTANCENAME"].str.split(";").str.len() != prod["ACTIVE_NUMERATOR_STRENGTH"].str.split(";").str.len()][["SUBSTANCENAME", "ACTIVE_NUMERATOR_STRENGTH"]].head()

,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH
49,NaN,NaN
58,NaN,NaN
59,NaN,NaN
60,NaN,NaN
69,NaN,NaN


Mostly NaN except...

In [78]:
prod[prod["SUBSTANCENAME"].str.split(";").str.len() != prod["ACTIVE_NUMERATOR_STRENGTH"].str.split(";").str.len()][["SUBSTANCENAME", "ACTIVE_NUMERATOR_STRENGTH"]].dropna()

,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH
90536,GLYCERIN; HYDROLYZED SOY PROTEIN (ENZYMATIC; 2...,10; .12


In [79]:
prod[prod["ACTIVE_NUMERATOR_STRENGTH"].str.split(";").str.len() != prod["ACTIVE_INGRED_UNIT"].str.split(";").str.len()][["ACTIVE_NUMERATOR_STRENGTH", "ACTIVE_INGRED_UNIT"]].dropna()

,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT


In [80]:
# drop the problematic object
prod = prod.drop(
    prod[prod["SUBSTANCENAME"].str.split(";").str.len() != prod["ACTIVE_NUMERATOR_STRENGTH"].str.split(";").str.len()][["SUBSTANCENAME", "ACTIVE_NUMERATOR_STRENGTH"]].dropna().index
)

While it does not mean that the number of classes matches the number of substances !

In [81]:
len(prod[prod["SUBSTANCENAME"].str.split(";").str.len() != prod["PHARM_CLASSES"].str.split(",").str.len()][ "PHARM_CLASSES"])

89302

Looking now at a possible OneHotEncoding approach to deal with multiple values per instance

In [82]:
print("ohe len categories for SubstanceName")
print(len(OneHotEncoder().fit(pd.DataFrame(prod["SUBSTANCENAME"].str.upper())).categories_[0]))
print("ohe len categories for ACTIVE_NUMERATOR_STRENGTH")
print(len(OneHotEncoder().fit(pd.DataFrame(prod["ACTIVE_NUMERATOR_STRENGTH"].str.upper())).categories_[0]))
print("ohe len categories for ACTIVE_INGRED_UNIT")
print(len(OneHotEncoder().fit(pd.DataFrame(prod["ACTIVE_INGRED_UNIT"].str.upper())).categories_[0]))

ohe len categories for SubstanceName
8949
ohe len categories for ACTIVE_NUMERATOR_STRENGTH
8737
ohe len categories for ACTIVE_INGRED_UNIT
2377


That amount of categories with OneHotEncoding might lead to overfitting, let's look at another avenue to deal with multiple values per instance those features

In [83]:
multiple_vals_feats = ["SUBSTANCENAME", "ACTIVE_NUMERATOR_STRENGTH", "ACTIVE_INGRED_UNIT"]

In [84]:
# Example with at least 2 multiple values in those features
prod[prod[multiple_vals_feats].apply(lambda x: x.str.split(";").str[1]).notnull().any(axis=1)][multiple_vals_feats]

,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT
11,OLANZAPINE; FLUOXETINE HYDROCHLORIDE,3; 25,mg/1; mg/1
12,OLANZAPINE; FLUOXETINE HYDROCHLORIDE,6; 25,mg/1; mg/1
13,OLANZAPINE; FLUOXETINE HYDROCHLORIDE,6; 50,mg/1; mg/1
14,OLANZAPINE; FLUOXETINE HYDROCHLORIDE,12; 50,mg/1; mg/1
121,ATAZANAVIR SULFATE; COBICISTAT,300; 150,mg/1; mg/1
...,...,...,...
93131,AVOBENZONE; OCTINOXATE; OCTISALATE; OCTOCRYLENE,.9; 1.3; 1.3; .6,g/30mL; g/30mL; g/30mL; g/30mL
93161,AVOBENZONE; HOMOSALATE; OCTISALATE; OCTOCRYLENE,.0306; .0816; .051; .0816,g/mL; g/mL; g/mL; g/mL
93162,MENTHOL; EUCALYPTUS OIL; CAMPHOR (SYNTHETIC),1; 1; 4.7,g/100g; g/100g; g/100g
93229,HYDROQUINONE; OXYBENZONE; PADIMATE O,20; 25; 33,mg/g; mg/g; mg/g


We could get all the element positions for the first 5 elements for each of the feature, meaning we would split in 15 new features

In [85]:
# Custom dummies-like method for first5
for i in range(5):
    # print([f"{feat}_{i}" for feat in multiple_vals_feats])
    prod[[f"{feat}_{i}" for feat in multiple_vals_feats]] = prod[multiple_vals_feats].apply(lambda x: x.str.split(";").str[i])

In [86]:
prod.isnull().sum()[19:]

SUBSTANCENAME_0                 2252
ACTIVE_NUMERATOR_STRENGTH_0     2252
ACTIVE_INGRED_UNIT_0            2252
SUBSTANCENAME_1                72009
ACTIVE_NUMERATOR_STRENGTH_1    72009
ACTIVE_INGRED_UNIT_1           72009
SUBSTANCENAME_2                81489
ACTIVE_NUMERATOR_STRENGTH_2    81489
ACTIVE_INGRED_UNIT_2           81489
SUBSTANCENAME_3                85985
ACTIVE_NUMERATOR_STRENGTH_3    85985
ACTIVE_INGRED_UNIT_3           85985
SUBSTANCENAME_4                88662
ACTIVE_NUMERATOR_STRENGTH_4    88662
ACTIVE_INGRED_UNIT_4           88662
dtype: int64

This is a lot of NaN but we will loose the link to pharm_classes if we only keep those. We have to choose a different approach like OneHotEncoding where the newfeature generated do not have NaN

In [87]:
prod = prod.drop(columns=prod.iloc[:, 19:].columns)

In [88]:
prod.head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH,ENDMARKETINGDATE_determined,DEASCHEDULE_determined,PROPRIETARYNAMESUFFIX_determined
0,NaN,0002-0800,HUMAN OTC DRUG,DILUENT,"INJECTION, SOLUTION",SUBCUTANEOUS,1987-07-10,NDA,NDA018781,10,WATER,1,mL/mL,NaN,N,2020-12-31,0,0,0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,FLORBETAPIR F 18,"INJECTION, SOLUTION",INTRAVENOUS,2012-06-01,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",N,2021-12-31,0,0,0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,DULAGLUTIDE,"INJECTION, SOLUTION",SUBCUTANEOUS,2014-09-18,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",N,2020-12-31,0,0,0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,DULAGLUTIDE,"INJECTION, SOLUTION",SUBCUTANEOUS,2014-09-18,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",N,2020-12-31,0,0,0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,GALCANEZUMAB,"INJECTION, SOLUTION",SUBCUTANEOUS,2018-09-27,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,N,2020-12-31,0,0,0


**PHARM_CLASSES Verification and correction**

In [89]:
print(f"There are {prod['PHARM_CLASSES'].isnull().sum()} missing values for PHARM_CLASSES")

There are 50921 missing values for PHARM_CLASSES


In [90]:
# Counts of Number of classes per objects
prod["PHARM_CLASSES"].str.split(",").str.len().sort_values().astype(str).value_counts()[:6]

nan    50921
2.0    23254
3.0     5039
4.0     3424
1.0     3073
5.0     3005
Name: PHARM_CLASSES, dtype: int64

In [91]:
prod["PHARM_CLASSES"].head()

0                                                  NaN
1    Radioactive Diagnostic Agent [EPC],Positron Em...
2    GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...
3    GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...
4                                                  NaN
Name: PHARM_CLASSES, dtype: object

**Extracting only the PHARM_CLASS code**

In [92]:
prod["PHARM_CLASSES"].head()

0                                                  NaN
1    Radioactive Diagnostic Agent [EPC],Positron Em...
2    GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...
3    GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...
4                                                  NaN
Name: PHARM_CLASSES, dtype: object

In [93]:
prod["PHARM_CLASSES"].str.findall("\[(.*?)\]").head()

0               NaN
1        [EPC, MoA]
2    [EPC, CS, MoA]
3    [EPC, CS, MoA]
4               NaN
Name: PHARM_CLASSES, dtype: object

There are repetitions within pharm_classes because it is linked to the SubstanceName molecules

In [94]:
prod["PHARM_CLASSES"].str.findall("\[(.*?)\]").dropna()[prod["PHARM_CLASSES"].dropna().str.findall("\[(.*?)\]").str.len() > prod["PHARM_CLASSES"].str.findall("\[(.*?)\]").dropna().apply(lambda x: set(x)).str.len()].head()

11    [EPC, EPC, MoA]
12    [EPC, EPC, MoA]
13    [EPC, EPC, MoA]
14    [EPC, EPC, MoA]
15    [MoA, EPC, MoA]
Name: PHARM_CLASSES, dtype: object

**NDC_Exclude_Flag codes verification**

In [95]:
ndc_exclude_codes = ["E", "U", "I"]
print([label for label in prod["NDC_EXCLUDE_FLAG"].unique()])

['N']


In [96]:
prod["NDC_EXCLUDE_FLAG"].value_counts()

N    93098
Name: NDC_EXCLUDE_FLAG, dtype: int64

Easy dropping that column since they are all the same value

In [97]:
prod = prod.drop(columns="NDC_EXCLUDE_FLAG")

**LISTING_RECORD_CERTIFIED_THROUGH dates verification**

In [98]:
print(f"Max time : {max(prod['LISTING_RECORD_CERTIFIED_THROUGH'])}")
print(f"Min time : {min(prod['LISTING_RECORD_CERTIFIED_THROUGH'])}")

Max time : 2021-12-31 00:00:00
Min time : 2020-12-31 00:00:00


This won't give any relevant information for a classification task. Also, it won't give more information than the start/end date which are more meaningful in terms of dating that just the end of certification date on the table. Obviously if we were assessing the validity of records at a certain time this feature would be important but for our classification task it is not. We can drop it

In [99]:
prod = prod.drop(columns="LISTING_RECORD_CERTIFIED_THROUGH")

In [100]:
prod.head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,ENDMARKETINGDATE_determined,DEASCHEDULE_determined,PROPRIETARYNAMESUFFIX_determined
0,NaN,0002-0800,HUMAN OTC DRUG,DILUENT,"INJECTION, SOLUTION",SUBCUTANEOUS,1987-07-10,NDA,NDA018781,10,WATER,1,mL/mL,NaN,0,0,0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,FLORBETAPIR F 18,"INJECTION, SOLUTION",INTRAVENOUS,2012-06-01,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",0,0,0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,DULAGLUTIDE,"INJECTION, SOLUTION",SUBCUTANEOUS,2014-09-18,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",0,0,0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,DULAGLUTIDE,"INJECTION, SOLUTION",SUBCUTANEOUS,2014-09-18,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",0,0,0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,GALCANEZUMAB,"INJECTION, SOLUTION",SUBCUTANEOUS,2018-09-27,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,0,0,0


### Exploring package data

In [101]:
pack.head()

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710,NaN,N,N
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601,NaN,N,N
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",20120601,NaN,N,N
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,20141107,NaN,N,Y
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-80,4 SYRINGE in 1 CARTON (0002-1433-80) > .5 mL ...,20141107,NaN,N,N


In [102]:
print(f"pack dataframe has {pack.shape[0]} objects and {pack.shape[1]} columns.") 

pack dataframe has 173887 objects and 8 columns.


In [103]:
print(f" There are {len(pack.dtypes[pack.dtypes != 'object'])} numerical columns :\n") 
print(pack.dtypes[pack.dtypes != "object"].index.to_list())

 There are 2 numerical columns :

['STARTMARKETINGDATE', 'ENDMARKETINGDATE']


In [104]:
print(f" There are {len(pack.dtypes[pack.dtypes == 'object'])} non-numerical columns :\n") 
print(pack.dtypes[pack.dtypes == "object"].index.to_list())

 There are 6 non-numerical columns :

['PRODUCTID', 'PRODUCTNDC', 'NDCPACKAGECODE', 'PACKAGEDESCRIPTION', 'NDC_EXCLUDE_FLAG', 'SAMPLE_PACKAGE']


**Evaluating NaN of all features**

In [105]:
pack.isnull().sum()

PRODUCTID                  0
PRODUCTNDC              1500
NDCPACKAGECODE          2346
PACKAGEDESCRIPTION         0
STARTMARKETINGDATE         0
ENDMARKETINGDATE      167431
NDC_EXCLUDE_FLAG           0
SAMPLE_PACKAGE             0
dtype: int64

**Evaluating PRODUCTNDC, PRODUCTID and NDCPACKAGECODE since they are interelated**

In [106]:
print(f"Num objects with no NA in ProdID/NDC : {len(pack[['PRODUCTID', 'PRODUCTNDC']].dropna())}")

Num objects with no NA in ProdID/NDC : 172387


In [107]:
print(f"Num objects with NDC code within ID col : {pack[['PRODUCTID', 'PRODUCTNDC']].dropna().apply(lambda x : x.PRODUCTNDC in x.PRODUCTID, axis=1).sum()}")

Num objects with NDC code within ID col : 171868


In [108]:
print(f"Num objects with incongruences btw ID and NDC : {len(pack[['PRODUCTID', 'PRODUCTNDC']].dropna()) - pack[['PRODUCTID', 'PRODUCTNDC']].dropna().apply(lambda x : x.PRODUCTNDC in x.PRODUCTID, axis=1).sum()}")

Num objects with incongruences btw ID and NDC : 519


Duplication within those 3 features

In [109]:
pack["PRODUCTNDC"].dropna().duplicated().sum()

81307

In [110]:
pack["PRODUCTID"].dropna().duplicated().sum()

80803

In [111]:
pack["NDCPACKAGECODE"].dropna().duplicated().sum()

94

Incongruency between the 3 features

In [112]:
# Looking at some of those objects
pack.iloc[pack[['PRODUCTID', 'PRODUCTNDC']].dropna()[pack[['PRODUCTID', 'PRODUCTNDC']].dropna().apply(lambda x : x.PRODUCTNDC not in x.PRODUCTID, axis=1)].index, :].tail()

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
36701,29300-111_4651c75e-b53c-4e27-ad2d-1e8af9111949,20111101,29300-111-01,"100 TABLET in 1 BOTTLE, PLASTIC (29300-111-01)",20111101,NaN,N,N
36702,29300-111_4651c75e-b53c-4e27-ad2d-1e8af9111949,20111101,29300-111-05,"500 TABLET in 1 BOTTLE, PLASTIC (29300-111-05)",20111101,NaN,N,N
36703,29300-111_4651c75e-b53c-4e27-ad2d-1e8af9111949,20111101,29300-111-10,"1000 TABLET in 1 BOTTLE, PLASTIC (29300-111-10)",20111101,NaN,N,N
36704,29300-112_4651c75e-b53c-4e27-ad2d-1e8af9111949,20111101,29300-112-01,"100 TABLET in 1 BOTTLE, PLASTIC (29300-112-01)",20111101,NaN,N,N
36705,29300-112_4651c75e-b53c-4e27-ad2d-1e8af9111949,20111101,29300-112-05,"500 TABLET in 1 BOTTLE, PLASTIC (29300-112-05)",20111101,NaN,N,N


In [113]:
pack[['NDCPACKAGECODE', 'PRODUCTNDC']].dropna()[pack[['NDCPACKAGECODE', 'PRODUCTNDC']].dropna().apply(lambda x : x.PRODUCTNDC not in x.NDCPACKAGECODE, axis=1)]


,NDCPACKAGECODE,PRODUCTNDC
228,0006-0005-13,05-juin
229,0006-0005-30,05-juin
230,0006-0019-54,19-juin
492,0009-0003-02,03-sept
493,0009-0005-01,05-sept
...,...,...
41910,20200531,37205-941
41913,20200831,37205-980
41914,20200831,37205-980
41928,20201130,37808-018


Let's drop those since they are mislabelled because they are suppose to be a match

In [114]:
pack = pack.drop(pack[['NDCPACKAGECODE', 'PRODUCTNDC']].dropna()[pack[['NDCPACKAGECODE', 'PRODUCTNDC']].dropna().apply(lambda x : x.PRODUCTNDC not in x.NDCPACKAGECODE, axis=1)].index)


That covers it

In [115]:
len(pack[['NDCPACKAGECODE', 'PRODUCTID']].dropna()[pack[['NDCPACKAGECODE', 'PRODUCTID']].dropna().apply(lambda x : x.NDCPACKAGECODE.rsplit("-", 1)[0] not in x.PRODUCTID, axis=1)])


0

All in all PRODUCTNDC contains the more mislabels so we can drop that column entirely since it will not be used in the merging of the 2 tables and classification task.
NDCPACKAGECODE won't be usefull for merging the two tables since it is absent from the product df

From the FDA website about PRODUCTID:
> 	ProductID is a concatenation of the NDCproduct code and SPL documentID. It is included to help prevent duplicate rows from appearing when joining the product and package files together.  It has no regulatory value or significance.

**We will use the ProductID to merge the 2 tables. we  can later verify if the product have mismatches after the join. Also we can drop the NDCPACKAGECODE now since it is redundant with the other 2 identification features**

In [116]:
pack = pack.drop(columns="NDCPACKAGECODE")

In [117]:
pack.head()

,PRODUCTID,PRODUCTNDC,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710,NaN,N,N
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601,NaN,N,N
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",20120601,NaN,N,N
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,20141107,NaN,N,Y
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,4 SYRINGE in 1 CARTON (0002-1433-80) > .5 mL ...,20141107,NaN,N,N


**PackageDescription contains some relevant information for a classification task such as volume, type of packaging. However, some information is redundant such as the NDCPACKAGECODE in parenthesis and some information such as the volume will be found in the Units used in the product table**

Let's try to extract at least the the size/type of the first occurence. We use regex to get the first occurence of size/type and a second capture group if `>` is present. We then only keep the first group or second based on weither it's present or not

In [118]:
pattern = r"^[^ ]+ ([^ ,]+)|> [^ ]+ ([^ ,\n]+)"

def extract_group(matches):
    if len(matches) == 2:
        return matches.iloc[1, 1]
    else:
        return matches.iloc[0, 0]

matches = pack["PACKAGEDESCRIPTION"].str.extractall(pattern)
# matches.groupby(level=0).apply(extract_group).values
# matches
results = matches.groupby(level=0).apply(extract_group)
pack["PACKAGEDESCRIPTION_FILTERED"] = results



In [119]:
pack[["PACKAGEDESCRIPTION", "PACKAGEDESCRIPTION_FILTERED"]].head(25)

,PACKAGEDESCRIPTION,PACKAGEDESCRIPTION_FILTERED
0,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,mL
1,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",mL
2,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",mL
3,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,mL
4,4 SYRINGE in 1 CARTON (0002-1433-80) > .5 mL ...,mL
5,2 SYRINGE in 1 CARTON (0002-1434-61) > .5 mL ...,mL
6,4 SYRINGE in 1 CARTON (0002-1434-80) > .5 mL ...,mL
7,1 SYRINGE in 1 CARTON (0002-1436-11) > 1 mL i...,mL
8,2 SYRINGE in 1 CARTON (0002-1436-61) > 1 mL i...,mL
9,3 SYRINGE in 1 CARTON (0002-1445-09) > 1 mL i...,mL


In [120]:
# Exploring the categories
print(f'We reduced to only {len(pack["PACKAGEDESCRIPTION_FILTERED"].unique())} categories for package description')

We reduced to only 83 categories for package description


In [121]:
pack = pack.drop(columns="PACKAGEDESCRIPTION")

**Start/End MarketingDate features will be redundant after the table join. For now let's just verify the logic and validity of the dates**

Note : NaN in EndMarketingDate means it has no end scheduled for now

In [122]:
pack.head()

,PRODUCTID,PRODUCTNDC,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE,PACKAGEDESCRIPTION_FILTERED
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,19870710,NaN,N,N,mL
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,20120601,NaN,N,N,mL
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,20120601,NaN,N,N,mL
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,20141107,NaN,N,Y,mL
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,20141107,NaN,N,N,mL


We have an error already when trying to change to datetime:

`pd.to_datetime(prod["STARTMARKETINGDATE"], format="%Y%m%d")`
`OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 2997-06-23 00:00:00`

In [123]:
# All samples with dates greater than 2024
print(f"Num objects with incoherent startdate : {len(pack[pack['STARTMARKETINGDATE'] > 20240000])}")
pack[pack["STARTMARKETINGDATE"] > 20240000].head()

Num objects with incoherent startdate : 243


,PRODUCTID,PRODUCTNDC,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE,PACKAGEDESCRIPTION_FILTERED
31,0002-4112_d561034d-ea58-45fe-9d07-2e9eba98c2e4,0002-4112,29970623,NaN,N,N,TABLET
110,0002-8501_7db61461-cf41-4de9-b5e6-3f88640d928c,0002-8501,29970106,NaN,N,N,mL
202,0004-0260_4b065b40-4aa6-440b-8120-82f7dbec615b,0004-0260,29970619,NaN,N,N,TABLET
203,0004-0260_4b065b40-4aa6-440b-8120-82f7dbec615b,0004-0260,29970619,NaN,N,N,TABLET
235,0006-0071_c24dac53-4c05-4e31-a42e-793efd0e1da4,0006-0071,29971219,NaN,N,N,TABLET


In [124]:
print(pack[pack["STARTMARKETINGDATE"] > 20240000]["STARTMARKETINGDATE"].unique())

[29970623 29970106 29970619 29971219 29970603 29971101 29970502 29970217
 29970801 29970516 29970313 29970513 29971224 29970311 29970312 29970130
 29970731 29971222 29971001 29970423 29970722 29970808 29971110 29970414
 29970803 29970918 29970515 29971007 29970228 29970829 29970605 29970930
 29970912 29970127 29970123 29971201 29971125 29971010 29970528 29971208
 29970729 29970207 29970805 29970901 29970212 29971218 29970929 29970326]


It seems like we need to substract 10000000 to the values to get them to the right year in 1997

In [125]:
# Replace faulty values
pack["STARTMARKETINGDATE"] = np.where(pack["STARTMARKETINGDATE"] > 20240000, pack["STARTMARKETINGDATE"] - 10000000, pack["STARTMARKETINGDATE"])
print(len(pack[pack['STARTMARKETINGDATE'] > 20240000]))

0


In [126]:
# Converto to datetime START
pack["STARTMARKETINGDATE"] = pd.to_datetime(pack["STARTMARKETINGDATE"], format="%Y%m%d")

Verification for ENDMARKETINGDATE

In [127]:
# Converto to datetime END
pack["ENDMARKETINGDATE"] = pd.to_datetime(pack["ENDMARKETINGDATE"], format="%Y%m%d")

In [128]:
print(f'There are {pack.isnull().sum()["ENDMARKETINGDATE"]} null in ENDMARKETINGDATE')

There are 166916 null in ENDMARKETINGDATE


In [129]:
# Over 75% of entries are null
na75_threshold = 0.75*len(pack)
print(pack.isnull().sum()[pack.isnull().sum() > na75_threshold].index.to_list())

['ENDMARKETINGDATE']


Let's convert to a binary since there is over 75% of null vals

In [130]:
pd.DataFrame(np.where(pack["ENDMARKETINGDATE"].notnull(), 1, 0)).value_counts()

0    166916
1      6297
dtype: int64

In [131]:
pack["ENDMARKETINGDATE_determined"] = np.where(pack["ENDMARKETINGDATE"].notnull(), 1, 0)
pack = pack.drop(columns="ENDMARKETINGDATE")

**NDC_EXCLUDE_FLAG verification**

In [132]:
# Counts
pack["NDC_EXCLUDE_FLAG"].value_counts()

N    173213
Name: NDC_EXCLUDE_FLAG, dtype: int64

Same as in the product table, a single value for every sample. Drop the column

In [133]:
pack = pack.drop(columns="NDC_EXCLUDE_FLAG")

**SAMPLE_PACKAGE verification + convert to binary**

In [134]:
# Possible values
pack["SAMPLE_PACKAGE"].value_counts()

N    172554
Y       659
Name: SAMPLE_PACKAGE, dtype: int64

Binary feature but not in the right type, let's convert to binary

In [135]:
pd.DataFrame(np.where(pack["SAMPLE_PACKAGE"] == "N", 0, 1)).head()

,0
0,0
1,0
2,0
3,1
4,0


In [136]:
pack["SAMPLE_PACKAGE"] = np.where(pack["SAMPLE_PACKAGE"] == "N", 0, 1)

---

## 4. Replacing missing values in prod and pack tables

**We will need to convert some of the categorical data to numerical data using get_dummies or onehotencoding in both tables**

### product table : 

1. Remove pharm_classes from table since it will be our classification task later on
2. Convert categorical data to proper format for imputation
3. Use different imputation techniques for each feature with missing values

In [137]:
# Removing the PHARM_CLASSES as y
y = prod["PHARM_CLASSES"]
prod = prod.drop(columns="PHARM_CLASSES")
y.head()

0                                                  NaN
1    Radioactive Diagnostic Agent [EPC],Positron Em...
2    GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...
3    GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...
4                                                  NaN
Name: PHARM_CLASSES, dtype: object

In [138]:
prod.head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,ENDMARKETINGDATE_determined,DEASCHEDULE_determined,PROPRIETARYNAMESUFFIX_determined
0,NaN,0002-0800,HUMAN OTC DRUG,DILUENT,"INJECTION, SOLUTION",SUBCUTANEOUS,1987-07-10,NDA,NDA018781,10,WATER,1,mL/mL,0,0,0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,FLORBETAPIR F 18,"INJECTION, SOLUTION",INTRAVENOUS,2012-06-01,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,0,0,0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,DULAGLUTIDE,"INJECTION, SOLUTION",SUBCUTANEOUS,2014-09-18,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,0,0,0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,DULAGLUTIDE,"INJECTION, SOLUTION",SUBCUTANEOUS,2014-09-18,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,0,0,0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,GALCANEZUMAB,"INJECTION, SOLUTION",SUBCUTANEOUS,2018-09-27,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,0,0,0


In [139]:
# semicolon separated entries
for col in prod.columns:
    if not col == "LABELERNAME":
        if prod[col].astype(str).str.contains("; ").sum() > 0:
            print(f'{col} feature contains {prod[col].astype(str).str.contains(";").sum()} entries with multiple elements')

ROUTENAME feature contains 4318 entries with multiple elements
SUBSTANCENAME feature contains 21089 entries with multiple elements
ACTIVE_NUMERATOR_STRENGTH feature contains 21089 entries with multiple elements
ACTIVE_INGRED_UNIT feature contains 21089 entries with multiple elements


Converting multiple values to list for each feature

In [302]:
prod["ROUTENAME"].str.split(";").str.len().value_counts()

1.0    86894
2.0     2224
3.0     1846
4.0      188
5.0       48
6.0        9
9.0        2
7.0        1
Name: ROUTENAME, dtype: int64

We can filter for routename with 3 elements only and put the rest in another category

In [303]:
prod["ROUTENAME_list"] = prod["ROUTENAME"].str.split("; ")

In [304]:
prod[prod["ROUTENAME_list"].isnull()].head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,ENDMARKETINGDATE_determined,DEASCHEDULE_determined,PROPRIETARYNAMESUFFIX_determined,ROUTENAME_list
69,0002-8031_f43b949f-3a0a-43bf-863d-4de52abe5faf,0002-8031,HUMAN PRESCRIPTION DRUG,GLUCAGON,KIT,NaN,1999-03-01,NDA,NDA020928,ELI LILLY AND COMPANY,NaN,NaN,NaN,0,0,0,NaN
123,0003-3764_507c9bab-1385-4e6b-95c1-fa8578a8a901,0003-3764,HUMAN PRESCRIPTION DRUG,APIXABAN,KIT,NaN,2017-11-29,NDA,NDA202155,"E.R. SQUIBB & SONS, L.L.C.",NaN,NaN,NaN,0,0,1,NaN
149,0004-0381_0b811e50-d0e0-4679-8d14-d9c91e867334,0004-0381,HUMAN PRESCRIPTION DRUG,ENFUVIRTIDE,KIT,NaN,2012-04-10,NDA,NDA021481,"GENENTECH, INC.",NaN,NaN,NaN,0,0,0,NaN
222,0006-3862_137e0347-9e23-4cc0-a44a-8c88d8983e17,0006-3862,HUMAN PRESCRIPTION DRUG,APREPITANT,KIT,NaN,2003-03-26,NDA,NDA021549,MERCK SHARP & DOHME CORP.,NaN,NaN,NaN,0,0,0,NaN
264,0006-5424_7ef0f037-02b3-417c-a736-23b9e9c6dd5a,0006-5424,PLASMA DERIVATIVE,LATRODECTUS MACTANS,KIT,NaN,2014-12-01,BLA,BLA101062,MERCK SHARP & DOHME CORP.,NaN,NaN,NaN,0,0,0,NaN


In [306]:
# Replace NaN with [UNKNOWN]
# prod["ROUTENAME_list"] = prod["ROUTENAME_list"].fillna("").apply(lambda x: ["UNKNOWN"] if x == "" else x)

In [305]:
prod["ROUTENAME_list"].value_counts().head()

[ORAL]                                       54716
[TOPICAL]                                    21557
[INTRAVENOUS]                                 3291
[INTRADERMAL, PERCUTANEOUS, SUBCUTANEOUS]     1315
[RESPIRATORY (INHALATION)]                    1257
Name: ROUTENAME_list, dtype: int64

In [308]:
# Drop routename
prod = prod.drop(columns="ROUTENAME")

Transforming multiple labels to list for 3 linked attributes : substancename, Activte_NUMERATOR_STRENGHT, Activte_UNIT

In [312]:
prod["SUBSTANCENAME_list"] = prod["SUBSTANCENAME"].str.split("; ")
prod["ACTIVE_NUMERATOR_STRENGTH_list"] = prod["ACTIVE_NUMERATOR_STRENGTH"].str.split("; ")
prod["ACTIVE_INGRED_UNIT_list"] = prod["ACTIVE_INGRED_UNIT"].str.split("; ")

In [315]:
# Dropping the ori feature
prod = prod.drop(columns=["SUBSTANCENAME", "ACTIVE_NUMERATOR_STRENGTH", "ACTIVE_INGRED_UNIT"])

In [316]:
prod.isnull().sum()

PRODUCTID                            1560
PRODUCTNDC                              0
PRODUCTTYPENAME                         0
NONPROPRIETARYNAME                      4
DOSAGEFORMNAME                          0
STARTMARKETINGDATE                      0
MARKETINGCATEGORYNAME                   0
APPLICATIONNUMBER                   13097
LABELERNAME                             0
ENDMARKETINGDATE_determined             0
DEASCHEDULE_determined                  0
PROPRIETARYNAMESUFFIX_determined        0
ROUTENAME_list                       1886
SUBSTANCENAME_list                   2252
ACTIVE_NUMERATOR_STRENGTH_list       2252
ACTIVE_INGRED_UNIT_list              2252
dtype: int64

In [321]:
for col in prod.columns:
    if prod[col].astype(str).str.contains(",").sum() > 0:
        print(f'{col} feature contains {prod[col].astype(str).str.contains(",").sum()} entries with multiple elements')

NONPROPRIETARYNAME feature contains 14814 entries with multiple elements
DOSAGEFORMNAME feature contains 27739 entries with multiple elements
APPLICATIONNUMBER feature contains 4 entries with multiple elements
LABELERNAME feature contains 34549 entries with multiple elements
ROUTENAME_list feature contains 4318 entries with multiple elements
SUBSTANCENAME_list feature contains 21450 entries with multiple elements
ACTIVE_NUMERATOR_STRENGTH_list feature contains 21089 entries with multiple elements
ACTIVE_INGRED_UNIT_list feature contains 21089 entries with multiple elements


This one is a bit tricky. I'll just leave it as is and labelencode it later since some are comma separated and some are not

In [328]:
prod[prod["NONPROPRIETARYNAME"].str.split(",").str.len() > 2]["NONPROPRIETARYNAME"]

221      IMIPENEM ANHYDROUS, CILASTATIN, AND RELEBACTAM...
228      HUMAN PAPILLOMAVIRUS QUADRIVALENT (TYPES 6, 11...
229             ROTAVIRUS VACCINE, LIVE, ORAL, PENTAVALENT
234      HUMAN PAPILLOMAVIRUS QUADRIVALENT (TYPES 6, 11...
237      MEASLES, MUMPS, RUBELLA AND VARICELLA VIRUS VA...
                               ...                        
93130    AVOBENZONE, OCTINOXATE, OCTISALATE, AND OCTOCR...
93131    AVOBENZONE, OCTINOXATE, OCTISALATE, AND OCTOCR...
93161    AVOBENZONE, HOMOSALATE, OCTISALATE, AND OCTOCR...
93162                     MENTHOL, CAMPHOR, EUCALYPTUS OIL
93229             HYDROQUINONE, OXYBENZONE, AND PADIMATE O
Name: NONPROPRIETARYNAME, Length: 10370, dtype: object

We can remove the 4 ApplicationNumber outliers

In [329]:
prod["APPLICATIONNUMBER"].dropna()[prod["APPLICATIONNUMBER"].dropna().str.contains(",")]

64347    part356,part355
64348    part356,part355
76899    part341,part348
85113    part341,part348
Name: APPLICATIONNUMBER, dtype: object

In [330]:
prod = prod.drop(prod["APPLICATIONNUMBER"].dropna()[prod["APPLICATIONNUMBER"].dropna().str.contains(",")].index)

Let's convert DosageFormName

In [335]:
prod["DOSAGEFORMNAME"].str.split(", ").str.len().value_counts()

1    65355
2    25001
3     1955
4      783
Name: DOSAGEFORMNAME, dtype: int64

In [336]:
# Convert and drop ori feature
prod["DOSAGEFORMNAME_list"] = prod["DOSAGEFORMNAME"].str.split(", ")


In [338]:
prod = prod.drop(columns="DOSAGEFORMNAME")

---

Convert categorical features to labels and perform KNN-imputation on NaN data

In [339]:
prod.isnull().sum()

PRODUCTID                            1560
PRODUCTNDC                              0
PRODUCTTYPENAME                         0
NONPROPRIETARYNAME                      4
STARTMARKETINGDATE                      0
MARKETINGCATEGORYNAME                   0
APPLICATIONNUMBER                   13097
LABELERNAME                             0
ENDMARKETINGDATE_determined             0
DEASCHEDULE_determined                  0
PROPRIETARYNAMESUFFIX_determined        0
ROUTENAME_list                       1886
SUBSTANCENAME_list                   2248
ACTIVE_NUMERATOR_STRENGTH_list       2248
ACTIVE_INGRED_UNIT_list              2248
DOSAGEFORMNAME_list                     0
dtype: int64

In [341]:
prod_features = []

In [340]:
prod.dtypes

PRODUCTID                                   object
PRODUCTNDC                                  object
PRODUCTTYPENAME                             object
NONPROPRIETARYNAME                          object
STARTMARKETINGDATE                  datetime64[ns]
MARKETINGCATEGORYNAME                       object
APPLICATIONNUMBER                           object
LABELERNAME                                 object
ENDMARKETINGDATE_determined                  int64
DEASCHEDULE_determined                       int64
PROPRIETARYNAMESUFFIX_determined             int64
ROUTENAME_list                              object
SUBSTANCENAME_list                          object
ACTIVE_NUMERATOR_STRENGTH_list              object
ACTIVE_INGRED_UNIT_list                     object
DOSAGEFORMNAME_list                         object
dtype: object

In [129]:
# prod[prod.duplicated("PRODUCTNDC", keep=False)].sort_values(by="PRODUCTNDC")[:10]

In [265]:
# Multilabel encoder fit
mlb = MultiLabelBinarizer()
mlb.fit(prod["ROUTENAME_list"])

MultiLabelBinarizer()

In [272]:
mlb_labels = [f"ROUTENAME_{label}" for label in mlb.classes_]
print(mlb_labels[:10])

['ROUTENAME_AURICULAR (OTIC)', 'ROUTENAME_BUCCAL', 'ROUTENAME_CUTANEOUS', 'ROUTENAME_DENTAL', 'ROUTENAME_ELECTRO-OSMOSIS', 'ROUTENAME_ENDOTRACHEAL', 'ROUTENAME_ENTERAL', 'ROUTENAME_EPIDURAL', 'ROUTENAME_EXTRACORPOREAL', 'ROUTENAME_HEMODIALYSIS']


In [271]:
# Join the MultiLabelEncoder output to prod df
prod = prod.join(pd.DataFrame(mlb.transform(prod["ROUTENAME_list"].dropna()), index=prod.index, columns=mlb_labels))

In [ ]:
prod.drop(columns=[]"ROUTENAME", "ROUTENAME")